In [13]:
import pandas as pd

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas,  numpy, textblob, string
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

In [3]:
df=pd.read_excel('AI Dataset.xlsx')

In [4]:
df.shape

(2309, 2)

In [5]:
df.head()

,TAG,COMMENT
0,Positive Project Feedback,I am feeling satisfied with the project
1,Positive Supervisor Feedback,You make me so happy. It’s like your superpower.
2,Positive Supervisor Feedback,You make me laugh. You make me smile. You make...
3,Positive Supervisor Feedback,The supervisor help the students a lot
4,Positive Supervisor Feedback,Thinking about you gives me a bad case of the ...


In [9]:
df['TAG'].unique()

array(['Positive Project Feedback', 'Positive Supervisor Feedback',
       'Positive Group Feedback', 'Negative Group Feedback',
       'Negative Supervisor Feedback', 'Negative Project Feedback',
       'Budget', 'Positive MPU4 Committee Feedback',
       'Negative MPU4 Committee Feedback', 'Collaborator'], dtype=object)

In [8]:
len(df['TAG'].unique())

10

In [14]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['COMMENT'], df['TAG'])

In [45]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['COMMENT'].values.astype('U'))

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x.values.astype('U'))
xvalid_count =  count_vect.transform(valid_x.values.astype('U'))


# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)

tfidf_vect.fit(df['COMMENT'].values.astype('U'))
xtrain_tfidf =  tfidf_vect.transform(train_x.values.astype('U'))
xvalid_tfidf =  tfidf_vect.transform(valid_x.values.astype('U'))


# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['COMMENT'].values.astype('U'))
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x.values.astype('U'))
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x.values.astype('U'))

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['COMMENT'].values.astype('U'))
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x.values.astype('U')) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x.values.astype('U')) 

In [46]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [30]:
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

NB, WordLevel TF-IDF:  0.4359861591695502


In [47]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print( "NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.47923875432525953
NB, WordLevel TF-IDF:  0.4359861591695502
NB, N-Gram Vectors:  0.3944636678200692
NB, CharLevel Vectors:  0.4204152249134948


In [48]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LR, Count Vectors:  0.4757785467128028
LR, WordLevel TF-IDF:  0.47923875432525953
LR, N-Gram Vectors:  0.3806228373702422
LR, CharLevel Vectors:  0.47750865051903113


In [63]:

classifier=linear_model.LogisticRegression()
    # fit the training dataset on the classifier
classifier.fit(xtrain_tfidf, train_y)
# predict the labels on validation dataset


  


C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
xvalid_tfidf[0]

<1x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [ ]:
predictions = classifier.predict(xvalid_tfidf)

In [62]:
predictions

array(['Positive Group Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Negative Project Feedback',
       'Negative Project Feedback', 'Positive Group Feedback',
       'Negative Project Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Negative Project Feedback',
       'Negative Project Feedback', 'Negative Project Feedback',
       'Positive Group Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Positive Group Feedback',
       'Negative Project Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Negative Project Feedback',
       'Negative Project Feedback', 'Negative Project Feedback',
       'Positive Group Feedback', 'Positive Group Feedback',
       'Positive Group Feedback', 'Positive Group Feedback',
    

In [50]:
classifier.predict(feature_vector_valid)

864            Negative Project Feedback
279              Negative Group Feedback
229              Negative Group Feedback
1177        Negative Supervisor Feedback
1826             Positive Group Feedback
190                         Collaborator
2100           Positive Project Feedback
1397             Positive Group Feedback
845            Negative Project Feedback
1064           Negative Project Feedback
546              Negative Group Feedback
1339             Positive Group Feedback
1207             Positive Group Feedback
1123           Negative Project Feedback
1078           Negative Project Feedback
549              Negative Group Feedback
1928             Positive Group Feedback
1353             Positive Group Feedback
1148           Negative Project Feedback
2273        Positive Supervisor Feedback
2074           Positive Project Feedback
1008           Negative Project Feedback
808            Negative Project Feedback
1251             Positive Group Feedback
842            N

In [32]:
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LR, WordLevel TF-IDF:  0.47923875432525953


In [35]:
accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print("SVM, wordlevel Vectors: ", accuracy)

C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, wordlevel Vectors:  0.3477508650519031


In [37]:
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

C:\Users\mateen.ahmad\AppData\Roaming\Python\Python36\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, WordLevel TF-IDF:  0.4290657439446367


In [39]:
import xgboost

In [40]:
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:58:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Xgb, WordLevel TF-IDF:  0.4515570934256055
